## Generation of the GT for Training

In [1]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
# export_ds = '/media/scratch/berlukas/nuscenes'
sem_clouds_filename = f"{export_ds}/clouds1.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /mnt/data/datasets/nuscenes/processed/clouds1.npy
Shape of sem clouds is (11230, 3, 200, 200)


In [ ]:
n_clouds = sem_cloud_features.shape[0]
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:,1]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
n_unique_classes = len(np.unique(classes))
print(f"we have {n_unique_classes} classes in the dataset.")

In [3]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 2
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (3, -1)).T    
    classes[i] = sem_cloud[:,idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
n_unique_classes = len(np.unique(classes))
print(f"we have {n_unique_classes} classes in the dataset.")


shape of classes is (11230, 40000)
we have 33 classes in the dataset.


## Iterate over the features and generate the GT

In [4]:
def map_nuscenes_label(label):
    noise = []
    living = [1,2,3,4,5,6,7,8]
    vehicle = [14,15,16,17,18,19,20,21,22,23]
    objects = [9,10,11,12,13]
    ground = [24,25,26,27]
    buildings = [28]
    vegetation = [30]
    other = [29,0]
    ego = [31]
    
    if label in noise:
        return 0
    elif label in vehicle:
        return 1
    elif label in living:
        return 2
    elif label in objects:
        return 3
    elif label in ground:
        return 4
    elif label in buildings:
        return 5
    elif label in vegetation:
        return 6
    elif label in other:
        return 7
    elif label in ego:
        return 8
    else:
        return 0    
    
def map_nuscenes_label_16(label):                       
    noise = [1, 5, 6, 8, 10, 11, 13, 19, 20, 0, 29, 31]
    barrier = [9]
    bicycle = [14]
    bus = [15, 16]
    car = [17]
    construction_vehicle = [18]
    motorcycle = [21]
    pedestrian = [2, 3, 4, 6]
    traffic_cone = [12]
    trailer = [22]
    truck = [23]
    driveable_surface = [24]
    other_flat = [25]
    sidewalk = [26]
    terrain = [27]
    manmade = [28]
    vegetation = [30]
        
    if label in noise:
        return 0
    elif label in barrier:
        return 1
    elif label in bicycle:
        return 2
    elif label in bus:
        return 3
    elif label in car:
        return 4
    elif label in construction_vehicle:
        return 5
    elif label in motorcycle:
        return 6
    elif label in pedestrian:
        return 7
    elif label in traffic_cone:
        return 8
    elif label in trailer:
        return 9
    elif label in truck:
        return 10
    elif label in driveable_surface:
        return 11
    elif label in other_flat:
        return 12
    elif label in sidewalk:
        return 13
    elif label in terrain:
        return 14
    elif label in manmade:
        return 15
    elif label in vegetation:
        return 16    
    else:
        return 0

In [ ]:
n_features = sem_cloud_features.shape[0]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]
idx_sem = 1

reshaped_gt = np.zeros((n_features, n_rows, n_cols))
for feature in tqdm(range(n_features)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = map_nuscenes_label(cur_class)
            mapped_class = map_nuscenes_label(cur_class)
            reshaped_gt[feature, i, j] = mapped_class

In [5]:
n_data = sem_cloud_features.shape[0]
n_features = sem_cloud_features.shape[1]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]

for feature in tqdm(range(n_data)):
    for i in range(n_rows):
        for j in range(n_cols):
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = map_nuscenes_label(cur_class)
            mapped_class = map_nuscenes_label_16(cur_class)
            sem_cloud_features[feature, idx_sem, i, j] = mapped_class

In [6]:
sem_clouds_gt_filename = f'{export_ds}/sem_clouds1_16.npy'
# np.save(sem_clouds_gt_filename, reshaped_gt)
np.save(sem_clouds_gt_filename, sem_cloud_features)
print(f"Wrote features to {sem_clouds_gt_filename}")

Wrote features to /mnt/data/datasets/nuscenes/processed/sem_clouds1_16.npy


In [7]:
# Remove a few samples for validation
n_clouds = sem_cloud_features.shape[0]
print(f'we have in total {n_clouds}')
n_train = n_clouds - 6053
n_val = n_clouds - n_train
print(f'we will keep {n_train} for training and {n_val} for validation.')

print(f'init shape is {sem_cloud_features.shape}')
sem_train = sem_cloud_features[:n_train, :, :, :]
sem_val = sem_cloud_features[n_train:, :, :, :]
print(f'training shape is {sem_train.shape}, validation shape is {sem_val.shape}')

np.save(f'{export_ds}/sem_clouds3_16.npy', sem_train)
np.save(f'{export_ds}/sem_clouds_val_16.npy', sem_val)

we have in total 11467
we will keep 5414 for training and 6053 for validation.
init shape is (11467, 3, 200, 200)
training shape is (5414, 3, 200, 200), validation shape is (6053, 3, 200, 200)
